<a href="https://colab.research.google.com/github/egonik-unlp/tp_curso_ml/blob/main/src/notebooks/autoencoder_minibatch_128_filtros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import requests
import json
import os
from datetime import datetime
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import normalize, MinMaxScaler
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
codigo_numerico=requests.get('https://raw.githubusercontent.com/egonik-unlp/tp_curso_ml/d61d2cd55fe79750c892a739129de1e02ead2d29/src/pickle/json_files/codigo_blosum62.json').json()

In [ ]:
%cd /content/drive/MyDrive/protein_classifier_with_locations/autoencoder/dataset
proteome=np.load("proteome_short.npy")

/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/dataset


In [ ]:
X_train_val, X_test=train_test_split(proteome)
X_train, X_valid=train_test_split(X_train_val)

# Por el momento voy a tratar de escalar manualmente, para evitar problemas de redondeo en la
# conversión inversa

X_train_n=X_train/24 # el valor maximo del 'codigo numerico'
X_valid_n=X_valid/24
X_test_n=X_test/24

In [ ]:
# Reshapeo para que funcione en el AE
X_train_bis=X_train_n.reshape(-1,2000,1)
X_valid_bis=X_valid_n.reshape(-1,2000,1)
X_test_bis=X_test_n.reshape(-1,2000,1)
del X_train_n
del X_valid_n
del X_test_n

In [ ]:
tf.keras.backend.clear_session()

conv_encoder = keras.models.Sequential([
keras.layers.Reshape([2000, 1], input_shape=[2000,1]),
keras.layers.Conv1D(16, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2),
keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2),
keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2),
keras.layers.Conv1D(128, kernel_size=3, padding='same', activation="selu"),
keras.layers.MaxPool1D(pool_size=2)
])


conv_decoder = keras.models.Sequential([
keras.layers.Conv1DTranspose(64, kernel_size=3,strides=2,padding='same',
                             input_shape=[125,128]),

keras.layers.Conv1DTranspose(32, kernel_size=3, strides=2, padding="same",
activation="selu",
#input_shape=[250, 64]
),
keras.layers.Conv1DTranspose(16, kernel_size=3, strides=2, padding="same",
activation="selu"),
keras.layers.Conv1DTranspose(1, kernel_size=3, strides=2, padding="same",
activation="sigmoid"),

])


conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])
conv_ae.summary()
datetime_string=datetime.now().strftime("%d-%m_%H-%M")
folder=dir='/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/sequences_checkpoints/runs/{}'.format(datetime_string)
os.mkdir(folder)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 250, 64)           7840      
_________________________________________________________________
sequential_1 (Sequential)    (None, 2000, 1)           7777      
Total params: 15,617
Trainable params: 15,617
Non-trainable params: 0
_________________________________________________________________


In [ ]:
nums_codes={v:k for k,v in codigo_numerico.items()}

In [ ]:
def unpad(query, target):
  try:
    idx=np.where(query==0)[0][-1] + 1
  except IndexError:
    idx=0
  return query[idx:], target[idx:]
class CustomSaver(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if epoch %50 == 0:  # or save after some epoch, each k-th epoch etc.
      self.model.save('{}/autoencoder_prot_epoch_{}.h5'.format(folder,epoch))
cs=CustomSaver()
  
class ProteinEvaluator(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}, protein_code=nums_codes,folder=folder):
    if epoch%5 == 0 and epoch!=0:
      predictions=self.model.predict(X_valid_bis)
      pred_seqs=[]
      for n,(query, target) in enumerate(zip(X_valid_bis.reshape(-1,2000), predictions.reshape(-1,2000))):
        qry,tgt=unpad(query,target)
        del qry
        pred_seqs.append(tgt)
      compressed_proteins=[''.join(np.vectorize(lambda x:nums_codes.get(round(x*25),'X'))(seq)) for seq in pred_seqs]
      with open('{}/pred_seqs_epoch_{}.json'.format(folder,epoch), 'w') as file:
        json.dump(compressed_proteins, file)
# pe=ProteinEvaluator()

In [ ]:
np.save('{}/X_valid_bis'.format(folder), X_valid_bis)
np.save('{}/X_test_bis'.format(folder), X_test_bis)
np.save('{}/X_train_bis'.format(folder), X_train_bis)

In [ ]:
conv_ae.compile(loss="binary_crossentropy",
# optimizer=keras.optimizers.SGD(learning_rate=1.5))
optimizer=keras.optimizers.Adam())
history = conv_ae.fit(X_train_bis, X_train_bis, epochs=1000,
  validation_data=(X_valid_bis, X_valid_bis), callbacks=[cs], batch_size=256)

Epoch 1/1000
373/373 [==============================] - 81s 67ms/step - loss: 0.1446 - val_loss: 0.1161
Epoch 2/1000
373/373 [==============================] - 23s 61ms/step - loss: 0.1124 - val_loss: 0.1085
Epoch 3/1000
373/373 [==============================] - 23s 61ms/step - loss: 0.1062 - val_loss: 0.1043
Epoch 4/1000
373/373 [==============================] - 23s 61ms/step - loss: 0.1036 - val_loss: 0.1027
Epoch 5/1000
373/373 [==============================] - 23s 62ms/step - loss: 0.1026 - val_loss: 0.1019
Epoch 6/1000
373/373 [==============================] - 23s 62ms/step - loss: 0.1019 - val_loss: 0.1014
Epoch 7/1000
373/373 [==============================] - 23s 62ms/step - loss: 0.1014 - val_loss: 0.1009
Epoch 8/1000
373/373 [==============================] - 23s 62ms/step - loss: 0.1009 - val_loss: 0.1005
Epoch 9/1000
373/373 [==============================] - 23s 62ms/step - loss: 0.1006 - val_loss: 0.1002
Epoch 10/1000
373/373 [==============================] - 23s 62m

In [ ]:
original_seqs=[seq for seq in X_test_bis.reshape(-1,2000)]

In [ ]:
# nums_codes={v:k for k,v in codigo_numerico.items()} 
# # compressed_proteins=[''.join(np.vectorize(lambda x:nums_codes.get(round(x*25),'X'))(seq)) for seq in pred_seqs]
# original_proteins=[''.join(np.vectorize(lambda x:nums_codes.get(round(x*25),'X'))(seq)) for seq in original_seqs]

In [ ]:
# np.save('/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/predictions',predictions)
# np.save('/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/compressed_proteins',compressed_proteins)
# np.save('/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/original_proteins', original_proteins)

In [ ]:
conv_ae.save("{}/model.h5".format(folder))

In [ ]:
pd.DataFrame(history.history).to_excel("{}/history.xlsx".format(folder))